In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [64]:
# Dataset is now stored in a Pandas Dataframe
Data = pd.read_csv('/kaggle/input/hamoye-stage-c/Data_for_UCI_named.csv')

In [65]:
Data.head(5)

In [66]:
Data.columns

In [67]:
Data.info()

In [68]:
Data.isnull().sum()

In [69]:
Data = Data.drop('stab', axis=1)

In [70]:
Data.columns

In [71]:
Data.head()

In [72]:
Data.nunique()

In [73]:
# Generate a correlation map of the data
plt.figure(figsize=(20,10))
sns.heatmap(Data.corr(), annot=True, cmap='plasma')
plt.show()

In [74]:
#splitting features and target variable
x = Data.drop('stabf', axis=1)
y = Data['stabf']

### MODELLING AND PREPROCESSING

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=1)
scaler = StandardScaler() 
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_test = scaler.transform(x_test)

In [76]:
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_train = pd.DataFrame(scaled_x_train,columns = x_train.columns)
scaled_x_test = scaler.transform(x_test)
scaled_x_test = pd.DataFrame(scaled_x_test, columns = x_test.columns)

In [77]:
from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier(random_state=1)

In [78]:
#fit the random forest model to the training dataset
rfr.fit(scaled_x_train, y_train)
#obtain predictions
predicted_values = rfr.predict(scaled_x_test)

In [79]:
# Obtain the accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predicted_values)
round(accuracy, 4) 

In [80]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state = 1)
#fit the extra trees model to the training dataset
etc.fit(scaled_x_train, y_train)
#obtain predictions
predicted_values = etc.predict(scaled_x_test)

In [81]:
# Obtain the accuracy
accuracy = accuracy_score(y_test, predicted_values)
round(accuracy, 4) 

In [82]:
from xgboost import XGBClassifier
xgbc = XGBClassifier(random_state=1)
#fit the xgboost model to the training dataset
xgbc.fit(scaled_x_train, y_train)
#obtain predictions
predicted_values = xgbc.predict(scaled_x_test)

In [83]:
# Obtain the accuracy
accuracy = accuracy_score(y_test, predicted_values)
round(accuracy, 4) 

In [84]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=1)
#fit the lightgbm model to the training dataset
lgbm.fit(scaled_x_train, y_train)
#obtain predictions
predicted_values = lgbm.predict(scaled_x_test)

In [85]:
# Obtain the accuracy
accuracy = accuracy_score(y_test, predicted_values)
round(accuracy, 4)

In [86]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [50,100,300,500,1000] 
min_samples_split = [2,3,5,7,9] 
min_samples_leaf = [1,2,4,6,8] 
max_features = ['auto','sqrt','log2',None] 
hyperparameter_grid = {'n_estimators':n_estimators,
                       'min_samples_leaf':min_samples_leaf,
                       'min_samples_split':min_samples_split,
                       'max_features':max_features}


In [87]:
rand_search = RandomizedSearchCV(estimator = etc,param_distributions=hyperparameter_grid,random_state=1)
rand_search = rand_search.fit(scaled_x_train,y_train)

In [88]:
rand_search.best_params_

In [89]:
n_classifier = ExtraTreesClassifier(random_state=1,n_estimators = 1000,
                                       min_samples_leaf= 8,max_features=None,
                                       min_samples_split=2)
n_classifier.fit(scaled_x_train,y_train)
predicted_values = n_classifier.predict(scaled_x_test)

In [90]:
# Obtain the accuracy
accuracy = accuracy_score(y_test, predicted_values)
round(accuracy, 4)

In [91]:
imp = rand_search.best_estimator_.feature_importances_
feat_imp = pd.DataFrame()
feat_imp['feature'] = x.columns
feat_imp['importance'] = imp
feat_imp.head()

In [92]:
feat_imp.sort_values(by='importance', ascending=False)

### Merci Beaucoup